In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-text-classification/ecommerceDataset.csv


In [7]:
df = pd.read_csv("/kaggle/input/ecommerce-text-classification/ecommerceDataset.csv",names= ["Label","Description"],header = None)
df.head()

,Label,Description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


# drop NA

In [10]:
df.isna().sum()

Label          0
Description    1
dtype: int64

In [12]:
df.dropna(inplace=True)

In [13]:
df.Label.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [17]:
df.Label.replace('Clothing & Accessories','Clothing_Accessories', inplace = True)

In [18]:
df.Label.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

### fasttext model,  expects labels to be specified with label prefix. We will just create a third column in the dataframe that has label as well as the product description

In [19]:
df["Label"] = "__label__" + df["Label"].astype(str)

In [20]:
df.head()

,Label,Description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [23]:
df["Description_Label"] =df["Label"] +" "+ df["Description"].astype(str)

In [24]:
df.head()

,Label,Description,Description_Label
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [25]:
df.Description[0]

'Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some for eternal blis

In [32]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [33]:
df["Description_Label"] =df["Description_Label"].map(preprocess)

In [34]:
df.head()

,Label,Description,Description_Label
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


# Train n Test  N spilt

In [35]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [36]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [39]:
train.to_csv("ecommerce.train", columns=["Description_Label"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["Description_Label"], index=False, header=False)

# Train the model and evaluate performance

In [58]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train",lr=0.7,epoch=10)

Read 4M words
Number of words:  79107
Number of labels: 4
Progress: 100.0% words/sec/thread: 1805754 lr:  0.000000 avg.loss:  0.052359 ETA:   0h 0m 0s


In [59]:
model.test("ecommerce.test")

(10085, 0.9799702528507684, 0.9799702528507684)

### First parameter (10084) is test size. Second and third parameters are precision and recall respectively. We can see we are getting around 97% precision which is pretty good

In [68]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99684161]))

In [69]:
model.predict("Open Library is an open, editable library catalog, building towards a web page for every book ever published. Read, borrow, and discover more than 3M books")

(('__label__books',), array([1.00000942]))